In [1]:
%load_ext autoreload
%autoreload 2
import main
import storage
import ranks
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import itertools
from itertools import takewhile,dropwhile
from fractions import Fraction
import pd_cols

c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df = storage.merge_and_load()
# df=df[~(df['note'] == 'ForcedSpecimen0')]
# storage.overwrite(df)
# 42

In [3]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2
import storage, ranks
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import itertools
from itertools import takewhile,dropwhile
from fractions import Fraction
import pd_cols
import math
import scipy.stats
import scipy
def p(func, /, *args, **keywords):
    def newfunc(*fargs, **fkeywords):
        newkeywords = {**keywords, **fkeywords}
        return func(*args, *fargs, **newkeywords)
    newfunc.func = func
    newfunc.args = args
    newfunc.keywords = keywords
    return newfunc

if False: #delete the all experiment records
        shutil.rmtree("ppdata")
        shutil.rmtree("exdata")
        shutil.rmtree("graphs")

df_og = storage.merge_and_load()
df_og = df_og[(df_og['note']=='') | (df_og['note']=='none') ]
df_og['full_desc'] = df_og.apply(pd_cols.get_full_desc,axis=1)
df_og = ranks.compute_ranks(df_og)

# helper functions and axilliary data 
pd.options.mode.chained_assignment = None          # prevents displaying a useless warning

def df_enhance(df):
        # df['true_evaluations'] = (df['pop_size'] * df['true_ratio']).map(int)
        df['gen_mult'] = df['gen_mult'].map(int) 
        df['dim_red'] = df['dim_red'].replace('','none')
        df['model'] = df['model'].replace('','none')
        df['pop_size'] = df['pop_size'].replace('None','none')
        df['actual_pop_size'] = df.apply(lambda r: r['pop_size'] if r['pop_size'] != 'none' else 4 + math.floor(3 * math.log(r['dim'])),axis=1)
        df['dim_red_kind'] = df['dim_red'].map(lambda a: (''.join(takewhile(lambda s: s.isalpha(), a))).lower())
        df['model_kind'] = df['model'].map(lambda a: (''.join(takewhile(lambda s: s.isalpha(), a))).lower())
        df['surrogate'] = df['model'].map(lambda a: (''.join(takewhile(lambda s: s.isalnum(), a))).lower())
        # df['spearman_corr'] = df['spearman_corr'].map(np.nan_to_num)
        # df['spearman_pval'] = df['spearman_pval'].map(np.nan_to_num)
        return df

df_og = df_enhance(df_og)
# df_og = df_og[df_og['scale_train']]
pure_mask = df_og['gen_mult'].map(int) == 1
pures = df_og[pure_mask] 
# pures2 = df_og[df_og['scale_train']==Tru] 

# pca_mask = (df_og['model_kind'] == 'gp')&(df_og['dim_red_kind'] == 'pca')&((df_og['pop_size']==48)|(df_og['pop_size']==64))&(df_og['true_ratio'].map(Fraction)==Fraction(1/8))
# pca_df = df_og[pca_mask] 

# keep the special cases separate, it makes all the graphing easier
df_og = df_og[~(
       (pure_mask&(df_og['pop_size'] != 'none'))
    #    |(pca_mask&(df_og['dim_red'] != 'pca0.5'))   
)]  




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# df = storage.merge_and_load()
# df['model_kind'] = df['model'].map(lambda a: (''.join(takewhile(lambda s: s.isalpha(), a))).lower())
# # df[df['model_kind']=='doe']['dists'] = [d[-len(c):] for d,c in  zip(df[df['model_kind']=='doe']['dists'].to_list(),df[df['model_kind']=='doe']['correct_invariant'].to_list())]
# df['dists'] = df.apply(lambda r: r['dists'][-len(r['correct_invariant']):] if r['model_kind']=='doe' else r['dists'],axis=1)
# print()
# storage.overwrite(df)

In [5]:
# import matplotlib.pyplot as plt
import scipy
def xy_scatter(xs, ys, len_mod_s, len_mod_e, xdesc, ydesc):
    fig, ax = plt.subplots()
    if xs is 'index':
        xs, ys = list(zip(*[(i,item) for arr in  xs.to_list() for (i,item) in enumerate(arr[int(len(arr)*len_mod_s):int(len(arr)*len_mod_e)])]))
    else:
        xs = np.array([item for arr in  xs.to_list() for item in arr[int(len(arr)*len_mod_s):int(len(arr)*len_mod_e)]])
        ys = np.array([item for arr in  ys.to_list() for item in arr[int(len(arr)*len_mod_s):int(len(arr)*len_mod_e)]])
   
    
    not_nan = ~(np.isnan(xs)|np.isnan(ys))
    xs, ys=np.array(xs)[not_nan], np.array(ys)[not_nan]
    ax.scatter(xs, ys, marker='.')
    
    # m, b = np.polyfit(dists, corr_invariant_tracker, 1)
    lr = scipy.stats.linregress(xs, ys)
    xx = np.linspace(np.min(xs), np.max(xs), num=100)
    ax.plot(xx, lr.slope*xx + lr.intercept,color='red')
    txt = [
        "r^2 = {:.3f}".format(lr.rvalue**2),
        'linreg start: {:.3f}'.format(lr.intercept),
        'linreg end: {:.3f}'.format(lr.slope*np.max(xs) + lr.intercept),

    ]
    ax.annotate('\n'.join(txt), (0.8*np.max(xs), 0.8*np.max(ys)))
    ax.set_xlabel(xdesc.title())
    ax.set_ylabel(ydesc.title())
    fig.show()


 <>:5: SyntaxWarning:"is" with a literal. Did you mean "=="?
 <>:5: SyntaxWarning:"is" with a literal. Did you mean "=="?
 C:\Users\Admin\AppData\Local\Temp\ipykernel_4344\1961603376.py:5: SyntaxWarning:"is" with a literal. Did you mean "=="?


In [6]:
def two_layer_tics(ax):
        plt.xticks(rotation=0, size= 'xx-small')
        for tick in ax.xaxis.get_major_ticks()[1::2]:
                tick.set_pad(15)
def save_and_show(name:str, show=True):
        plt.savefig(f'graphs/{name}.png', bbox_inches='tight')
        if show: plt.show()
def print_latex(df: pd.Series):
        r = lambda s: ' ' if r is None else s.replace('_', ' ')
        s= "\\begin{tabular}{|lc|}\n"  # + " | ".join(["c"] * len(df.columns)) + "}\n"
        s+= "\\hline\n"
        s+= r(df.index.name) + ' & ' + r(df.name) +'\\\\\n'
        s+= "\\hline\n"
        for k, v in df.items():
                s+= f"{k} & {v:0.2f} \\\\\n"
        s+= "\\hline\n"
        s+= "\\end{tabular}"
        print(s)
def np_apply_axis0(fn=None):
        def inner(arr, fn):
                arr = arr.to_list()
                b = np.apply_along_axis(fn,0,arr) 
                return list(b)
        return lambda a: inner(a, fn)
def avg_axis0_rugged(arr):
        arr = arr.to_list()
        m_l = max(map(len, arr))
        masks = []
        padded = []
        for a in arr:
                to_pad = m_l-len(a)
                mask = [True]*len(a) + [False]*(to_pad)
                p_arr = np.pad(a,(0, to_pad), constant_values=0)   
                padded.append(p_arr)
                masks.append(mask)
        stacked= np.stack(padded, axis=0)
        num_valid_in_each_col = np.sum(np.array(masks),axis=0)
        avg = np.sum(stacked, 0)/num_valid_in_each_col
        return list(avg)

def close_to(series, num):
        return series.map(lambda a: abs(a - num) <= 1e-3)

def get_param_desc_title(df):
    title_stringer = lambda beginning, name: beginning + ' ' + str(df[name].min()) + (('-'+str(df[name].max())) if df[name].min() != df[name].max() else '')
    title = title_stringer('fun','function') + '; dim ' + ', '.join([str(a) for a in np.unique(df['dim'])]) + title_stringer('; inst','instance')
    return title

def default_groupby(df, columns):
        map_dict = {
                'ranks':np_apply_axis0(np.average), 
                'avg_rank':'mean', 
                'last_rank':'mean',
                'elapsed_time':'mean',
                'model':'first',
                'dim_red':'first',
                'model_kind':'first',
                'dim_red_kind':'first',
                'gen_mult':'first'
        }
        for c in columns:  
               if c in map_dict: 
                      del map_dict[c]
        res = df.groupby(columns).agg(map_dict)
        return res

baselines = default_groupby(pures, ['pop_size'])
baseline_color = '#E04836'
default_color = 'forestgreen'
def bar(df,x_name=None, y_name='avg_rank',  index_mapper = None, y_mapper = None, regr = False, baseline_i=-1, x_ticklabel_mapper=None, print_table=True, title = ''):
        if x_name != None:    
            df = default_groupby(df,x_name)
        df = df.sort_index()
        
        if print_table is not None:
                print(print_table)
                print_latex(df[y_name])
        colors = [default_color for _ in range(len(df))]
        if baseline_i != -1 :
                colors += [baseline_color]
                df.loc[str(len(df))] = baselines.loc[baseline_i]
                
        x = df.index
        y = df[y_name] 
        
        # if index_mapper != None: 
        #        x = x.map(index_mapper)
        # if y_mapper != None: 
        #        y = y.map(y_mapper)
        ax = sns.barplot(x=x.map(str), y=y, palette=colors)
        # fig, ax = plt.subplots()
        # ax.bar(x.map(str), y, label=bar_labels, color=bar_colors)
        ax.set_ylabel('Rank Percentile')
        xn = df.index.name if x_name == None else x_name
        if xn != None: # df.index.name can be None
                xn = xn.replace('_', ' ').title()
                ax.set_xlabel(xn)
        xticklabels = ax.get_xticklabels()
        if x_ticklabel_mapper:
               xticklabels = x_ticklabel_mapper(xticklabels)
        if baseline_i != -1 :
                xticklabels[-1] = 'baseline'
                plt.axhline(y=y[-1], color=baseline_color, linestyle='dotted')
        ax.set_xticklabels(xticklabels, size='small')
        # from pydoc import locate
# >>> locate('int')
        # print('\n'.join(map(lambda (a,b): f'{a}',zip(y,x))))
        if regr:
                xx = np.arange(len(y)-(1 if baseline_i != -1  else 0))
                m, b = np.polyfit(xx, y[:-1] if baseline_i != -1  else y, 1)
                ax.plot(xx, m*xx + b,color='red', alpha=0.5)
        ax.title = title + '\n' + get_param_desc_title(df)
        return ax 



In [3]:

gen_lims = [10, 4, 2]
dims = [2,5,10, None]
fn = None
func_groups = [
    (1,5, 'Separable Functions'),
    (6,9, 'Functions with low or moderate conditioning'),
    (10,14, 'Ill conditioned functions'),
    (15,19, 'Adequately structured multimodal functions'),
    (20,24, 'Weakly structured multimodal functions'),
    (1,24, 'All functions')
]
for eval_limit in gen_lims:
    for dim in dims:
        for (s,e,description) in func_groups:
            df = df_og.copy()
            df = df[(df['pop_size']=='none')]
            title = []
            
            if description != '':
                # title.append(f'func {s}-{e}')
                title.append(description)
                df = df[(df['function']>=s)&(df['function']<=e)]
            if dim != None:
                title.append(f'dim {dim}')
                df = df[(df['dim']==dim)]
            else: title.append(f'all dims')
            
            if eval_limit != 1:
                a = eval_limit
                title.append(f'first 1/{a} generations')
                eval_limit = int(50/eval_limit)
            title = ', '.join(title)

            # df = df[(df['note']=='')|((df['gen_mult']==1)&(df['pop_size']=='none'))]
            import math
            # df['best_per_sett']
            df['improvement_percent'] = df['vals'].apply(lambda a: 100*(1-(a-a[-1])/(a[3]-a[-1])))
            df['convergence_cutoff'] = df['improvement_percent'].apply(lambda a: np.argmin(a>99.99))
            # df = df[df['convergence_cutoff']>5]
            # df['evals'] = df.apply(lambda r: r['evals'][:r['convergence_cutoff']],axis=1)
            # df['vals'] = df.apply(lambda r: r['vals'][:r['convergence_cutoff']],axis=1)
            

            # df = df[(df['model']=='doe_32_8')&(df['dim']==2)]
            # xy_scatter(df['dists'],df['spearman_corr'], 0, 1, 'latent space euklidean distance', 'spearman rank correlation')
            # xy_scatter(df,'doe_16_4', 0,1, 'dists', 'spearman_pval', 'latent space euklidean distance', 'spearman rank p-value')

            df['vals'] = df.apply(lambda r: r['vals'][:eval_limit],axis=1)
            df['evals'] = df.apply(lambda r: r['evals'][:eval_limit],axis=1)
            df = ranks.compute_ranks(df, eval_limit=eval_limit)
            df['surrogate'] = df['model'].map(lambda a: (''.join(takewhile(lambda s: s.isalnum(), a))).lower())
            dfg = default_groupby(df, 'surrogate')
            # dfg['avg_rank'] = dfg['ranks'].apply(np.mean)
            # dfg['median_rank'] = dfg['ranks'].apply(np.median)
            ax = bar(dfg,y_name='avg_rank', print_table=title)
            plt.title(title)
            save_and_show(title.replace('/', ''))
            # plt.show()
            # for tick in ax.xaxis.get_major_ticks()[1::2]:
            #     tick.set_pad(15)
            # plt.xticks(size= 25)
            # aaaa = avg_axis0_rugged(df['ranks'])
            # 

            # for s,e,description in func_groups:
            #     df1 = df[(df['function']>=s)&(df['function']<=e)]
            #     ax = bar(dfg,y_name='last_rank')
            #     for tick in ax.xaxis.get_major_ticks()[1::2]:
            #         tick.set_pad(15)
            #     plt.xticks(size= 5)



NameError: name 'df_og' is not defined

In [ ]:
df = df_og.copy()
df = df[((df['note']=='pop scale, dim scale')|((df['gen_mult']==1))&(df['pop_size']=='none'))&(df['dim']==10)]
ranks.coco_plot(df)

In [7]:
df = df_og
df = df[(df['approx_dist']!= -1)&(df['dim']== 20)]
print(df['approx_dist'].describe())


KeyError: 'approx_dist'

In [ ]:
df = df_og
df = df[(df['approx_dist']!= -1)&(df['dim']!= 40)]
print(df['approx_dist'].describe())

In [ ]:
df = df_og
ax = bar(df,'full_desc',)
for tick in ax.xaxis.get_major_ticks()[1::2]:
    tick.set_pad(15)
plt.xticks(size= 5)
save_and_show('rrrrr')

In [ ]:
df = df_og
df = default_groupby(df, ['true_ratio'])
ax = bar(df,'true_ratio')

save_and_show('rrrrr')

In [ ]:
df = pures
# df = df[df['pop_size']]
ax = bar(df, 'pop_size')
ax.set_xlabel('Population Size')
ax.set_title('Normal Evaluation')

save_and_show('pure')

In [ ]:
# pca 
from itertools import takewhile,dropwhile

st =False
df=pca_df[pca_df['scale_train']==st]
df = df[(df['model_kind'] == 'gp')&(df['pop_size']==64)]

df['pca_ratio'] = df['dim_red'].map(lambda a: ''.join(dropwhile(lambda s: s.isalpha(), a)))

# df1 = df_og[(df_og['model_kind'] == 'gp')&(df_og['dim_red_kind'] == 'none')&(df_og['pop_size']==48)&(df_og['true_ratio'].map(Fraction)==Fraction(1/8))].iloc[0]
# df1['pca_ratio'] = str(1.0)
# df.loc[str(len(df))] = df1
df1 = df_og[(df_og['model_kind'] == 'gp')&(df_og['dim_red_kind'] == 'none')&(df_og['pop_size']==64)&(df_og['true_ratio'].map(Fraction)==Fraction(1/8))].iloc[0]

df1['pca_ratio'] = str(1.0)
df.loc[str(len(df))] = df1
ax = bar(df,'pca_ratio', regr=True, baseline_i=8)
ax.set_xlabel('pca reduction ratio')
ax.set_title('PCA + GP')

save_and_show('pca')

In [ ]:
df=df_og
df = df[(df['model_kind'] == 'gp')& (df['dim_red_kind'] == 'none')]  
df = default_groupby(df, ['true_evaluations', 'pop_size'])
pures2 = pures.set_index(pures['pop_size'].map(lambda n: (n,n)))
df = pd.concat([df, pures2])
ax = bar(df, print_table=False)

save_and_show('pop_evals')

In [ ]:
df=df_og
df = df[(df['model_kind'] == 'gp')]  
ax = bar(df, 'dim_red')
save_and_show('gp_dim_red')


In [ ]:
df=df_og
ax = bar(df, 'dim_red_kind', 'elapsed_time')
ax.set_ylabel('Iteration Time (ms)')
save_and_show('elapsed')

In [ ]:
df=df_og
df = df[(df['dim_red_kind'] == 'none')&(df['pop_size']==48)&(df['true_ratio'].map(Fraction)==Fraction(1/8))]  
ax = bar(df, 'model')
two_layer_tics(ax)
save_and_show('models')



In [ ]:
# dim_red
df=df_og
ax = bar(df, 'dim_red_kind')
save_and_show('dim_red_kind')


In [ ]:
# rank models without nodimred 
df=df_og
df1 = df[ (df['true_evaluations']==6) & (df['dim_red_kind'] == 'none')]  
ax = bar(df, 'model')
plt.xticks(rotation=90, size= 'small')
save_and_show('models')


In [ ]:
#popsize,true_evals           with only gp
from fractions import Fraction
df=df_og
df=df[(df['model_kind'] == 'gp')&(df['dim_red_kind']=='none')]

ax = bar(df, 'true_ratio',index_mapper = lambda a: Fraction(a))
ax.set_label('dim red, true evals, aux evals')
ax.set_xlabel('truly evaluated fraction of population')
ax.set_ylabel('rank percentile avg')

labels = [item.get_text() for item in ax.get_xticklabels()]
labels[1] = '1/12'
ax.set_xticklabels(labels)
save_and_show('popsize_evals')

In [2]:
#popsize,true_evals           with only gp
df=df_og
df=df[(df['model_kind'] == 'gp')&(df['dim_red_kind']=='none')]
df = default_groupby(df, ['true_evaluations','pop_size'])
ax = bar(df)
ax.set_label('dim red, true evals, aux evals')
ax.set_xlabel('population: (true evaluated, generated)')
ax.set_ylabel('rank percentile avg')
# xlabel = ax.get_xlabel()
# ax.set_xlabel([1,2,3], rotation='horizontal')
# plt.savefig("graphs/pop.png")
# plt.pause(0.01)
plt.show() 

NameError: name 'df_og' is not defined

In [65]:
# df=pca_df[(pca_df['scale_train']==True)&(pca_df['function']==8)&(pca_df['instance']==1)]
df = df_og
df =df[ 
    (df['dim']==2)&
    (df['function']<=5)
       
]
ranks.plot_ranks(df, 1)

 c:\Users\Admin\Desktop\Diplomka\ranks.py:160: FutureWarning:Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


In [1]:
df = df_og.copy()
df = df[df['note']=='']
out_folders = df['coco_directory'].unique()
np.array(out_folders)[True, ]
ranks.coco_plot(df)

NameError: name 'df_og' is not defined